In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

In [ ]:
!nvidia-smi

Thu Sep 30 04:18:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install --user opencv-python

In [ ]:
!mkdir testlocal
!unzip '/content/drive/MyDrive/iss/data/test.zip' -d trainlocal

Archive:  /content/drive/MyDrive/iss/data/test.zip
   creating: trainlocal/test/
   creating: trainlocal/test/fake/
  inflating: trainlocal/test/fake/00276TOPP4.jpg  
  inflating: trainlocal/test/fake/008BYSE725.jpg  
  inflating: trainlocal/test/fake/009ZTJ3621.jpg  
  inflating: trainlocal/test/fake/00F8LKY6JC.jpg  
  inflating: trainlocal/test/fake/00JEP4Z36Z.jpg  
  inflating: trainlocal/test/fake/00KEKJJ1Q4.jpg  
  inflating: trainlocal/test/fake/00MZYXAT77.jpg  
  inflating: trainlocal/test/fake/00PB1BNIE8.jpg  
  inflating: trainlocal/test/fake/00QKZTHTLF.jpg  
  inflating: trainlocal/test/fake/00V5CZZSSO.jpg  
  inflating: trainlocal/test/fake/00XUQJZGHU.jpg  
  inflating: trainlocal/test/fake/01050DBM3C.jpg  
  inflating: trainlocal/test/fake/010MNNNOZS.jpg  
  inflating: trainlocal/test/fake/017EMQKL4D.jpg  
  inflating: trainlocal/test/fake/01877XHF3A.jpg  
  inflating: trainlocal/test/fake/01EA4QAE1L.jpg  
  inflating: trainlocal/test/fake/01FCG0FF23.jpg  
  inflating: trai

In [ ]:
!unzip '/content/drive/MyDrive/iss/data/train.zip' -d trainlocal

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: trainlocal/train/real/63106.jpg  
  inflating: trainlocal/train/real/63108.jpg  
  inflating: trainlocal/train/real/63109.jpg  
  inflating: trainlocal/train/real/63110.jpg  
  inflating: trainlocal/train/real/63111.jpg  
  inflating: trainlocal/train/real/63113.jpg  
  inflating: trainlocal/train/real/63114.jpg  
  inflating: trainlocal/train/real/63115.jpg  
  inflating: trainlocal/train/real/63116.jpg  
  inflating: trainlocal/train/real/63117.jpg  
  inflating: trainlocal/train/real/63118.jpg  
  inflating: trainlocal/train/real/63119.jpg  
  inflating: trainlocal/train/real/63120.jpg  
  inflating: trainlocal/train/real/63121.jpg  
  inflating: trainlocal/train/real/63122.jpg  
  inflating: trainlocal/train/real/63124.jpg  
  inflating: trainlocal/train/real/63125.jpg  
  inflating: trainlocal/train/real/63126.jpg  
  inflating: trainlocal/train/real/63127.jpg  
  inflating: trainlocal/train/real/63128.jpg  
  inflating: trainlocal/train/rea

In [ ]:
!unzip '/content/drive/MyDrive/iss/data/valid.zip' -d trainlocal

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: trainlocal/valid/real/34825.jpg  
  inflating: trainlocal/valid/real/34827.jpg  
  inflating: trainlocal/valid/real/34828.jpg  
  inflating: trainlocal/valid/real/34832.jpg  
  inflating: trainlocal/valid/real/34836.jpg  
  inflating: trainlocal/valid/real/34839.jpg  
  inflating: trainlocal/valid/real/34847.jpg  
  inflating: trainlocal/valid/real/34848.jpg  
  inflating: trainlocal/valid/real/34852.jpg  
  inflating: trainlocal/valid/real/34861.jpg  
  inflating: trainlocal/valid/real/34864.jpg  
  inflating: trainlocal/valid/real/34868.jpg  
  inflating: trainlocal/valid/real/34877.jpg  
  inflating: trainlocal/valid/real/34883.jpg  
  inflating: trainlocal/valid/real/34887.jpg  
  inflating: trainlocal/valid/real/34891.jpg  
  inflating: trainlocal/valid/real/34894.jpg  
  inflating: trainlocal/valid/real/34904.jpg  
  inflating: trainlocal/valid/real/34905.jpg  
  inflating: trainlocal/valid/real/34911.jpg  
  inflating: trainlocal/valid/rea

In [ ]:
import cv2
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf

In [ ]:
def build_model(pretrained):
    model = Sequential([
        pretrained,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
"""
Plot the training and validation loss
epochs - list of epoch numbers
loss - training loss for each epoch
val_loss - validation loss for each epoch
"""
def plot_loss(epochs, loss, val_loss):
    plt.plot(epochs, loss, 'bo', label='Training Loss')
    plt.plot(epochs, val_loss, 'orange', label = 'Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()
    
    
"""
Plot the training and validation accuracy
epochs - list of epoch numbers
acc - training accuracy for each epoch
val_acc - validation accuracy for each epoch
"""
def plot_accuracy(epochs, acc, val_acc):
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'orange', label = 'Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.show()

In [ ]:
# base_path = '../combined-real-and-fake-faces/combined-real-vs-fake/'
base_path = '/content/trainlocal/'
image_gen = ImageDataGenerator()

train_flow = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(224, 224),
    batch_size=64,
    color_mode='grayscale',
    class_mode='binary'
)


Found 100000 images belonging to 2 classes.


In [ ]:
valid_flow = image_gen.flow_from_directory(
    base_path + 'valid/',
    target_size=(224, 224),
    batch_size=64,
    color_mode='grayscale',
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.


In [ ]:
test_flow = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(224, 224),
    batch_size=1,
    color_mode='grayscale',
    shuffle = False,
    class_mode='binary'
)

Found 632 images belonging to 1 classes.


In [ ]:
densenet = DenseNet121(
    weights=None,
    include_top=False,
    input_shape=(224,224,1)
)
model = build_model(densenet)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 7, 7, 1024)        7031232   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 7,032,257
Trainable params: 6,948,609
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
train_steps = 100000//64
valid_steps = 20000//64

history = model.fit(
    train_flow,
    epochs = 10,
    steps_per_epoch = train_steps,
    validation_data = valid_flow,
    validation_steps = valid_steps 
)

Epoch 1/10
1562/1562 [==============================] - 560s 358ms/step - loss: 0.3045 - accuracy: 0.8691 - val_loss: 0.5468 - val_accuracy: 0.7999
Epoch 2/10
1562/1562 [==============================] - 560s 359ms/step - loss: 0.1780 - accuracy: 0.9285 - val_loss: 0.3493 - val_accuracy: 0.8427
Epoch 3/10
1562/1562 [==============================] - 561s 359ms/step - loss: 0.1096 - accuracy: 0.9578 - val_loss: 0.1862 - val_accuracy: 0.9273
Epoch 4/10
1562/1562 [==============================] - 561s 359ms/step - loss: 0.0749 - accuracy: 0.9716 - val_loss: 0.1820 - val_accuracy: 0.9412
Epoch 5/10
1562/1562 [==============================] - 560s 358ms/step - loss: 0.0553 - accuracy: 0.9796 - val_loss: 0.1344 - val_accuracy: 0.9509
Epoch 6/10
1562/1562 [==============================] - 559s 358ms/step - loss: 0.0441 - accuracy: 0.9838 - val_loss: 0.1868 - val_accuracy: 0.9339
Epoch 7/10
1562/1562 [==============================] - 560s 358ms/step - loss: 0.0357 - accuracy: 0.9870 - val_

In [ ]:
model.save('mygrayscale_densenet.h5')

In [ ]:
model.save("/content/drive/MyDrive/mygrayscale_densenet.h5")

In [ ]:
model.load()


NameError: ignored

In [ ]:
model.predict("")